In [1]:
import pandas as pd
import numpy as np
from scipy.fft import fft, fftfreq
from scipy.signal import welch, find_peaks
from scipy.stats import kurtosis, skew
from sklearn.preprocessing import StandardScaler
from ripser import ripser
from multiprocessing import cpu_count
import concurrent.futures
import logging
import time

# Configure logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

class KeratometryAnalyzer:
    def __init__(self, file_path):
        """Initialize the analyzer with the data file path."""
        logging.debug(f"Initializing KeratometryAnalyzer with file: {file_path}")
        self.data = pd.read_csv(file_path)
        logging.debug(f"Data loaded. Shape: {self.data.shape}")
        self.preprocess_data()
        self.num_threads = cpu_count()
        self.fft_results = None
        self.welch_results = None
        logging.debug(f"Analyzer initialized with {self.num_threads} threads.")

    def preprocess_data(self):
        """Preprocess the data for analysis."""
        logging.debug("Starting data preprocessing.")
        self.data = self.data.sort_values(['Meridian_Index', 'Radial_Index'])
        if 'Meridian_Angle_Rad' not in self.data.columns:
            self.data['Meridian_Angle_Rad'] = np.deg2rad(self.data['Meridian_Angle_Deg'])
        logging.debug("Data preprocessing complete.")

    def _compute_fft(self):
        """Compute FFT for the entire dataset."""
        logging.debug("Starting FFT computation.")
        k_values = self.data['Keratometry_Value'].values
        fft_vals = fft(k_values)
        freqs = fftfreq(len(k_values))
        pos_mask = freqs > 0
        logging.debug("FFT computation complete.")
        return freqs[pos_mask], np.abs(fft_vals)[pos_mask]

    def _compute_welch(self):
        """Compute Welch's method for the entire dataset."""
        logging.debug("Starting Welch computation.")
        k_values = self.data['Keratometry_Value'].values
        result = welch(k_values, fs=1.0, nperseg=len(k_values)//4)
        logging.debug("Welch computation complete.")
        return result

    def fourier_metrics(self):
        """Calculate detailed metrics from Fourier analysis."""
        logging.debug("Starting fourier_metrics.")
        try:
            if self.fft_results is None:
                self.fft_results = self._compute_fft()

            freqs, fft_magnitude = self.fft_results

            # Find peaks
            peaks, properties = find_peaks(fft_magnitude, height=np.mean(fft_magnitude))
            peak_freqs = freqs[peaks]
            peak_magnitudes = fft_magnitude[peaks]

            metrics = {
                'dominant_frequency': peak_freqs[np.argmax(peak_magnitudes)],
                'dominant_magnitude': np.max(peak_magnitudes),
                'num_significant_peaks': len(peaks),
                'total_spectral_power': np.sum(fft_magnitude**2),
                'mean_magnitude': np.mean(fft_magnitude),
                'std_magnitude': np.std(fft_magnitude),
                'spectral_kurtosis': kurtosis(fft_magnitude),
                'spectral_skewness': skew(fft_magnitude),
                'peak_frequencies': peak_freqs[:5].tolist(),
                'peak_magnitudes': peak_magnitudes[:5].tolist(),
                'frequency_centroid': np.sum(freqs * fft_magnitude) / np.sum(fft_magnitude)
            }
            logging.debug("fourier_metrics complete.")
            return metrics
        except Exception as e:
            logging.error(f"Error in fourier_metrics: {str(e)}", exc_info=True)
            raise

    def harmonic_metrics(self):
        """Calculate detailed metrics from harmonic analysis."""
        logging.debug("Starting harmonic_metrics.")
        try:
            if self.welch_results is None:
                self.welch_results = self._compute_welch()
                
            frequencies, psd = self.welch_results
            
            # Find peaks
            peaks, properties = find_peaks(psd, height=np.mean(psd))
            peak_freqs = frequencies[peaks]
            peak_powers = psd[peaks]
            
            metrics = {
                'dominant_frequency': peak_freqs[np.argmax(peak_powers)],
                'max_power': np.max(peak_powers),
                'total_power': np.sum(psd),
                'mean_power': np.mean(psd),
                'power_std': np.std(psd),
                'power_kurtosis': kurtosis(psd),
                'power_skewness': skew(psd),
                'num_harmonics': len(peaks),
                'harmonic_frequencies': peak_freqs[:5].tolist(),
                'harmonic_powers': peak_powers[:5].tolist(),
                'power_bandwidth': np.sum(frequencies * psd) / np.sum(psd),
                'power_concentration': np.max(peak_powers) / np.sum(psd)
            }
            logging.debug("harmonic_metrics complete.")
            return metrics
        except Exception as e:
            logging.error(f"Error in harmonic_metrics: {str(e)}", exc_info=True)
            raise

    def spatial_metrics(self):
        """Calculate spatial metrics."""
        logging.debug("Starting spatial_metrics.")
        try:
            metrics = {
                'mean_k': np.mean(self.data['Keratometry_Value']),
                'std_k': np.std(self.data['Keratometry_Value']),
                'max_k': np.max(self.data['Keratometry_Value']),
                'min_k': np.min(self.data['Keratometry_Value']),
                'k_range': np.ptp(self.data['Keratometry_Value']),
                'radial_correlation': np.corrcoef(self.data['Normalized_Radius'], 
                                               self.data['Keratometry_Value'])[0,1],
                'angular_correlation': np.corrcoef(self.data['Meridian_Angle_Rad'], 
                                                self.data['Keratometry_Value'])[0,1],
                'spatial_kurtosis': kurtosis(self.data['Keratometry_Value']),
                'spatial_skewness': skew(self.data['Keratometry_Value'])
            }
            logging.debug("spatial_metrics complete.")
            return metrics
        except Exception as e:
            logging.error(f"Error in spatial_metrics: {str(e)}", exc_info=True)
            raise

    def _process_ripser_chunk(self, chunk):
        """Helper function to process ripser for a chunk of data."""
        logging.debug(f"Processing ripser chunk with size {len(chunk)}.")
        start_time = time.time()
        result = ripser(chunk, maxdim=2)['dgms']
        end_time = time.time()
        logging.debug(f"Rips chunk processed in {end_time - start_time:.2f} seconds.")
        return result
    
    def topological_metrics(self):
        """Calculate topological metrics using multithreading."""
        logging.debug("Starting topological_metrics.")
        try:
             features = self.data[['X_Coordinate', 'Y_Coordinate', 'Keratometry_Value']].values
             scaler = StandardScaler()
             scaled_features = scaler.fit_transform(features)
             logging.debug(f"Scaled features. Shape: {scaled_features.shape}")
            
             #Split Data into Chunks
             chunk_size = max(1, len(scaled_features) // self.num_threads)
             chunks = [scaled_features[i:i + chunk_size] for i in range(0, len(scaled_features), chunk_size)]
             logging.debug(f"Split data into {len(chunks)} chunks for ripser.")
             
             with concurrent.futures.ThreadPoolExecutor(max_workers=self.num_threads) as executor:
                futures = {executor.submit(self._process_ripser_chunk, chunk): chunk for chunk in chunks}
                
                # Collect results with timeout
                results = []
                for future in concurrent.futures.as_completed(futures, timeout=300): # 5 minute timeout
                    try:
                        results.extend(future.result())
                    except Exception as e:
                         logging.error(f"Error processing a ripser chunk: {str(e)}", exc_info=True)
                         raise

             logging.debug("All ripser chunks processed.")
             # Combine results and calculate metrics
             
             metrics = {}
             for dim, dgm in enumerate(results):
                persistence = dgm[:,1] - dgm[:,0]
                metrics.update({
                    f'dim_{dim}_num_features': len(persistence),
                    f'dim_{dim}_max_persistence': np.max(persistence) if len(persistence) > 0 else 0,
                    f'dim_{dim}_mean_persistence': np.mean(persistence) if len(persistence) > 0 else 0,
                    f'dim_{dim}_total_persistence': np.sum(persistence)
                    })
             logging.debug("topological_metrics complete.")
             return metrics
        except Exception as e:
            logging.error(f"Error in topological_metrics: {str(e)}", exc_info=True)
            raise

    def generate_report(self):
        """Generate a comprehensive report with all metrics using multithreading."""
        logging.debug("Starting generate_report.")
        try:
            with concurrent.futures.ThreadPoolExecutor(max_workers=self.num_threads) as executor:
                future_fourier = executor.submit(self.fourier_metrics)
                future_harmonic = executor.submit(self.harmonic_metrics)
                future_spatial = executor.submit(self.spatial_metrics)
                future_topological = executor.submit(self.topological_metrics)

                report = {
                    'Fourier_Analysis': future_fourier.result(timeout = 300), #5 min timeout
                    'Harmonic_Analysis': future_harmonic.result(timeout=300), #5 min timeout
                    'Spatial_Analysis': future_spatial.result(timeout=300), #5 min timeout
                    'Topological_Analysis': future_topological.result(timeout=300) #5 min timeout
                }
            # Save report to CSV
            report_df = pd.DataFrame()
            for category, metrics in report.items():
                temp_df = pd.DataFrame([metrics])
                temp_df.columns = [f'{category}_{col}' for col in temp_df.columns]
                report_df = pd.concat([report_df, temp_df], axis=1)

            report_df.to_csv('keratometry_analysis_report.csv', index=False)
            logging.debug("Report generation complete.")
            return report
        except Exception as e:
            logging.error(f"Error generating report: {str(e)}", exc_info=True)
            raise

def main():
    try:
        file_path = '/home/aricept094/mydata/testrm_converted.csv'
        
        print(f"\nInitializing analysis with {cpu_count()} CPU threads...")
        analyzer = KeratometryAnalyzer(file_path)
        
        print("\nGenerating report...")
        report = analyzer.generate_report()
        
        print("\nKey Metrics Summary:")
        print("-" * 50)
        
        print("\nFourier Analysis:")
        print(f"Dominant Frequency: {report['Fourier_Analysis']['dominant_frequency']:.4f}")
        print(f"Number of Significant Peaks: {report['Fourier_Analysis']['num_significant_peaks']}")
        print(f"Total Spectral Power: {report['Fourier_Analysis']['total_spectral_power']:.2f}")
        
        print("\nHarmonic Analysis:")
        print(f"Dominant Frequency: {report['Harmonic_Analysis']['dominant_frequency']:.4f}")
        print(f"Number of Harmonics: {report['Harmonic_Analysis']['num_harmonics']}")
        print(f"Power Concentration: {report['Harmonic_Analysis']['power_concentration']:.4f}")
        
        print("\nSpatial Analysis:")
        print(f"Mean Keratometry: {report['Spatial_Analysis']['mean_k']:.2f}")
        print(f"Radial Correlation: {report['Spatial_Analysis']['radial_correlation']:.4f}")
        print(f"Angular Correlation: {report['Spatial_Analysis']['angular_correlation']:.4f}")
        
        print("\nTopological Analysis:")
        print(f"Dimension 0 Features: {report['Topological_Analysis']['dim_0_num_features']}")
        print(f"Dimension 1 Features: {report['Topological_Analysis']['dim_1_num_features']}")
        
        print("\nFull report saved to 'keratometry_analysis_report.csv'")
    except Exception as e:
        logging.error(f"Error in main execution: {str(e)}", exc_info=True)
        raise

if __name__ == "__main__":
    main()

2025-01-06 17:29:39,074 - DEBUG - Initializing KeratometryAnalyzer with file: /home/aricept094/mydata/testrm_converted.csv
2025-01-06 17:29:39,089 - DEBUG - Data loaded. Shape: (8192, 10)
2025-01-06 17:29:39,090 - DEBUG - Starting data preprocessing.
2025-01-06 17:29:39,094 - DEBUG - Data preprocessing complete.
2025-01-06 17:29:39,095 - DEBUG - Analyzer initialized with 16 threads.
2025-01-06 17:29:39,096 - DEBUG - Starting generate_report.
2025-01-06 17:29:39,097 - DEBUG - Starting fourier_metrics.
2025-01-06 17:29:39,098 - DEBUG - Starting FFT computation.
2025-01-06 17:29:39,098 - DEBUG - Starting harmonic_metrics.
2025-01-06 17:29:39,099 - DEBUG - Starting spatial_metrics.
2025-01-06 17:29:39,100 - DEBUG - Starting topological_metrics.
2025-01-06 17:29:39,101 - DEBUG - Starting Welch computation.
2025-01-06 17:29:39,101 - DEBUG - FFT computation complete.
2025-01-06 17:29:39,114 - DEBUG - Scaled features. Shape: (8192, 3)
2025-01-06 17:29:39,114 - DEBUG - fourier_metrics complete.


Initializing analysis with 16 CPU threads...

Generating report...


2025-01-06 17:29:39,293 - DEBUG - Processing ripser chunk with size 512.
2025-01-06 17:29:39,338 - DEBUG - Processing ripser chunk with size 512.
2025-01-06 17:29:51,804 - DEBUG - Processing ripser chunk with size 512.
2025-01-06 17:29:51,805 - DEBUG - Rips chunk processed in 12.67 seconds.
2025-01-06 17:29:51,809 - DEBUG - Processing ripser chunk with size 512.
IOStream.flush timed out2025-01-06 17:29:51,814 - DEBUG - Processing ripser chunk with size 512.

IOStream.flush timed out
2025-01-06 17:29:51,817 - DEBUG - Processing ripser chunk with size 512.
2025-01-06 17:29:51,821 - DEBUG - Processing ripser chunk with size 512.
2025-01-06 17:30:01,389 - DEBUG - Rips chunk processed in 22.24 seconds.
IOStream.flush timed out
2025-01-06 17:30:35,952 - DEBUG - Rips chunk processed in 56.80 seconds.
2025-01-06 17:30:35,953 - DEBUG - Rips chunk processed in 56.82 seconds.
2025-01-06 17:31:06,445 - DEBUG - Rips chunk processed in 87.24 seconds.
2025-01-06 17:31:28,367 - DEBUG - Rips chunk proc


Key Metrics Summary:
--------------------------------------------------

Fourier Analysis:
Dominant Frequency: 0.0312
Number of Significant Peaks: 68
Total Spectral Power: 1165436451.64

Harmonic Analysis:
Dominant Frequency: 0.0312
Number of Harmonics: 16
Power Concentration: 0.4541

Spatial Analysis:
Mean Keratometry: 38.96
Radial Correlation: -0.9164
Angular Correlation: 0.0161

Topological Analysis:
Dimension 0 Features: 512
Dimension 1 Features: 356

Full report saved to 'keratometry_analysis_report.csv'


In [12]:
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
from scipy.fft import fft, ifft

# --- Load Data ---
file_path = "/home/aricept094/mydata/testrm_converted.csv"
try:
    data = pd.read_csv(file_path)
except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
    exit()

# --- Data Preprocessing ---

# Assuming your data has these columns:
# "Meridian_Index", "Radial_Index", "Meridian_Angle_Deg", "Meridian_Angle_Rad",
# "Normalized_Radius", "Cos_Theta", "Sin_Theta", "X_Coordinate", "Y_Coordinate",
# "Keratometry_Value"

# 1. Remove rows with missing values (if any)
data.dropna(
    subset=[
        "Meridian_Index",
        "Radial_Index",
        "Meridian_Angle_Deg",
        "Meridian_Angle_Rad",
        "Normalized_Radius",
        "Keratometry_Value",
    ],
    inplace=True,
)

# 2. Sort data for each radius (important for interpolation)
data.sort_values(by=["Radial_Index", "Meridian_Angle_Rad"], inplace=True)

# --- Fourier Analysis in Polar Coordinates ---

# --- Parameters ---
num_radial_points = 32  # Matching the Radial_Index in your data
num_angular_points = 256  # Matching the number of meridians
max_m = 15  # Maximum angular frequency to consider

# --- Storage for Metrics ---
metrics = {
    "A0": [],  # Average keratometry at each radius
    "Am": [],  # Cosine coefficients at each radius for each m
    "Bm": [],  # Sine coefficients at each radius for each m
    "Asymmetry_Index": [],  # Example of a derived metric
    "Regular_Astigmatism": [],  # Example of another derived metric
    "Radial_Distances": [],
}

# --- Loop through radial indices ---
unique_radial_indices = np.unique(data["Radial_Index"])




for radial_index in unique_radial_indices:
    # --- Extract data for the current radial index ---
    radial_data = data[data["Radial_Index"] == radial_index]

    # --- Get the radius corresponding to this index ---
    r = radial_data["Normalized_Radius"].iloc[0]

    if len(radial_data) < 2:
        continue

    # --- Interpolate to get evenly spaced angular data ---
    interp_func = interp1d(
        radial_data["Meridian_Angle_Rad"],
        radial_data["Keratometry_Value"],
        kind="cubic",
        fill_value="extrapolate",
    )
    theta_values = np.linspace(0, 2 * np.pi, num_angular_points, endpoint=False)
    keratometry_interp = interp_func(theta_values)

    # --- Perform FFT along the angular direction ---
    fft_result = fft(keratometry_interp)

    # --- Extract Fourier Coefficients ---
    A0 = np.real(fft_result[0]) / num_angular_points  # Average value
    Am = np.zeros(max_m)
    Bm = np.zeros(max_m)

    for m in range(1, max_m + 1):
        Am[m - 1] = 2 * np.real(fft_result[m]) / num_angular_points
        Bm[m - 1] = -2 * np.imag(fft_result[m]) / num_angular_points

    # --- Metrics Calculation (Examples) ---
    # 1. Asymmetry Index (example: ratio of sine to cosine energy)
    asymmetry_index = np.sum(Bm**2) / np.sum(Am**2) if np.sum(Am**2) != 0 else 0

    # 2. Regular Astigmatism (example: using m=2 coefficients)
    regular_astigmatism = np.sqrt(Am[1] ** 2 + Bm[1] ** 2)

    # --- Store Metrics ---
    metrics["A0"].append(A0)
    metrics["Am"].append(Am)
    metrics["Bm"].append(Bm)
    metrics["Asymmetry_Index"].append(asymmetry_index)
    metrics["Regular_Astigmatism"].append(regular_astigmatism)
    metrics["Radial_Distances"].append(r)
    metrics["J0"] = []
    metrics["J45"] = []
    metrics["Dominant_Frequency"] = []
    metrics["Dominant_Magnitude"] = []

    for i in range(len(metrics["Radial_Distances"])):
        # Power Vectors
        J0 = -metrics["Am"][i][1] / 2  # Using Am[2] which is at index 1 in the Am list
        J45 = -metrics["Bm"][i][1] / 2  # Using Bm[2] which is at index 1 in the Bm list

        # Dominant Frequency
        magnitudes = [
            np.sqrt(metrics["Am"][i][m] ** 2 + metrics["Bm"][i][m] ** 2)
            for m in range(max_m)
        ]
        dominant_m = np.argmax(magnitudes) + 1  # +1 because m starts from 1
        dominant_magnitude = magnitudes[dominant_m - 1]

        # Store
        metrics["J0"].append(J0)
        metrics["J45"].append(J45)
        metrics["Dominant_Frequency"].append(dominant_m)
        metrics["Dominant_Magnitude"].append(dominant_magnitude)

    # --- Output ---
    print("-----------------------------")
    for i in range(len(metrics["Radial_Distances"])):
        r = metrics["Radial_Distances"][i]
        print(f"  Radius (from Normalized_Radius): {r:.3f}")
        print(f"    A0: {metrics['A0'][i]:.3f}")
        for m in range(max_m):
            print(
                f"    Am[{m+1}]: {metrics['Am'][i][m]:.3f}, Bm[{m+1}]: {metrics['Bm'][i][m]:.3f}"
            )
        print(f"    Asymmetry Index: {metrics['Asymmetry_Index'][i]:.3f}")
        print(f"    Regular Astigmatism: {metrics['Regular_Astigmatism'][i]:.3f}")
        print(f"    J0: {metrics['J0'][i]:.3f}")
        print(f"    J45: {metrics['J45'][i]:.3f}")
        print(f"    Dominant Frequency (m): {metrics['Dominant_Frequency'][i]}")
        print(f"    Dominant Magnitude: {metrics['Dominant_Magnitude'][i]:.3f}")

-----------------------------
  Radius (from Normalized_Radius): 0.000
    A0: 44.916
    Am[1]: 0.008, Bm[1]: 0.015
    Am[2]: -1.646, Bm[2]: -0.204
    Am[3]: -0.010, Bm[3]: 0.034
    Am[4]: 0.066, Bm[4]: 0.013
    Am[5]: -0.015, Bm[5]: 0.004
    Am[6]: -0.034, Bm[6]: 0.022
    Am[7]: 0.003, Bm[7]: 0.001
    Am[8]: 0.001, Bm[8]: -0.003
    Am[9]: -0.001, Bm[9]: -0.001
    Am[10]: 0.002, Bm[10]: 0.000
    Am[11]: 0.000, Bm[11]: 0.000
    Am[12]: 0.001, Bm[12]: 0.001
    Am[13]: -0.001, Bm[13]: -0.001
    Am[14]: -0.002, Bm[14]: -0.000
    Am[15]: 0.000, Bm[15]: 0.001
    Asymmetry Index: 0.016
    Regular Astigmatism: 1.659
    J0: 0.823
    J45: 0.102
    Dominant Frequency (m): 2
    Dominant Magnitude: 1.659
-----------------------------
  Radius (from Normalized_Radius): 0.000
    A0: 44.916
    Am[1]: 0.008, Bm[1]: 0.015
    Am[2]: -1.646, Bm[2]: -0.204
    Am[3]: -0.010, Bm[3]: 0.034
    Am[4]: 0.066, Bm[4]: 0.013
    Am[5]: -0.015, Bm[5]: 0.004
    Am[6]: -0.034, Bm[6]: 0.022
 

In [38]:
import numpy as np 
from scipy.fft import fft2, ifft2, fftshift, ifftshift
import pandas as pd

class CornealFourierAnalyzer:
    def __init__(self, csv_path):
        """Initialize analyzer with corneal topography data."""
        self.data = pd.read_csv(csv_path)
        self.prepare_grid()
        self.compute_fft()
        
    def prepare_grid(self):
        """Prepare regular grid from polar data with proper normalization."""
        self.n_meridians = len(self.data['Meridian_Index'].unique())
        self.n_radial = len(self.data['Radial_Index'].unique())
        
        # Create empty grid
        self.keratometry_grid = np.zeros((self.n_meridians, self.n_radial))
        
        # Fill grid
        for _, row in self.data.iterrows():
            i = int(row['Meridian_Index']) - 1
            j = int(row['Radial_Index']) - 1
            self.keratometry_grid[i, j] = row['Keratometry_Value']
            
        # Store original grid before normalization
        self.original_grid = self.keratometry_grid.copy()
            
        # Normalize for FFT analysis
        self.grid_norm = np.sqrt(np.sum(np.abs(self.keratometry_grid)**2))
        if self.grid_norm > 0:
            self.keratometry_grid /= self.grid_norm
            
    def compute_fft(self):
        """Compute 2D FFT and related components."""
        # Compute 2D FFT with normalization
        self.fft_result = fft2(self.keratometry_grid)
        self.fft_shifted = fftshift(self.fft_result)
        
        # Compute magnitude and phase spectra
        self.magnitude_spectrum = np.abs(self.fft_shifted)
        self.phase_spectrum = np.angle(self.fft_shifted)
        self.power_spectrum = np.square(self.magnitude_spectrum)
        
        # Get DC component (centered after shift)
        self.dc_component = self.magnitude_spectrum[self.n_meridians//2, self.n_radial//2]

    def analyze_frequency_bands(self, n_bands=5):
        """Analyze frequency content in concentric bands."""
        center_y, center_x = self.n_meridians//2, self.n_radial//2 
        max_radius = min(center_y, center_x)
        bands = []
        
        for band in range(n_bands):
            inner_radius = (band * max_radius) / n_bands
            outer_radius = ((band + 1) * max_radius) / n_bands
            band_mask = np.zeros_like(self.magnitude_spectrum, dtype=bool)
            
            for i in range(self.magnitude_spectrum.shape[0]):
                for j in range(self.magnitude_spectrum.shape[1]):
                    distance = np.sqrt((i - center_y)**2 + (j - center_x)**2)
                    if inner_radius <= distance < outer_radius:
                        band_mask[i, j] = True
                        
            band_magnitudes = self.magnitude_spectrum[band_mask]
            band_power = self.power_spectrum[band_mask]
            
            if len(band_magnitudes) > 0:
                bands.append({
                    'band_index': band,
                    'frequency_range': (inner_radius, outer_radius),
                    'mean_magnitude': np.mean(band_magnitudes),
                    'max_magnitude': np.max(band_magnitudes),
                    'total_power': np.sum(band_power),
                    'relative_power': np.sum(band_power) / np.sum(self.power_spectrum) * 100,
                    'decay_rate': np.mean(band_magnitudes) / self.dc_component
                })
                
        return bands

    def get_top_components(self, n_components=20):
        """Get details of top frequency components."""
        # Flatten spectrum and get sorted indices
        flat_spectrum = self.magnitude_spectrum.flatten()
        sorted_indices = np.argsort(flat_spectrum)[::-1]
        
        total_power = np.sum(self.power_spectrum)
        components = []
        
        for idx in range(min(n_components, len(sorted_indices))):
            flat_idx = sorted_indices[idx]
            i, j = np.unravel_index(flat_idx, self.magnitude_spectrum.shape)
            
            # Convert to frequency coordinates (centered)
            freq_i = i - self.n_meridians//2
            freq_j = j - self.n_radial//2
            
            magnitude = self.magnitude_spectrum[i, j]
            power = self.power_spectrum[i, j]
            
            components.append({
                'rank': idx + 1,
                'meridional_freq': freq_i,
                'radial_freq': freq_j,
                'magnitude': magnitude,
                'phase': self.phase_spectrum[i, j],
                'power': power,
                'relative_power': (power / total_power) * 100,
                'cumulative_power': (np.sum([self.power_spectrum[np.unravel_index(
                    sorted_indices[k], self.magnitude_spectrum.shape)]
                    for k in range(idx + 1)]) / total_power) * 100
            })
            
        return components

    def get_reconstruction_formula(self, n_components=5):
        """Get reconstruction formula using top n components."""
        components = self.get_top_components(n_components)
        formula_parts = []
        
        for comp in components:
            freq_i = comp['meridional_freq'] 
            freq_j = comp['radial_freq']
            magnitude = comp['magnitude'] * self.grid_norm  # Denormalize
            phase = comp['phase']
            
            if comp['rank'] == 1:  # DC component
                term = f"{magnitude:.2f}"
            else:
                # Clear notation for meridional and radial components
                meridional_term = f"{freq_i}m/{self.n_meridians}"
                radial_term = f"{freq_j}r/{self.n_radial}"
                
                term = (f"{magnitude:.2f} * cos(2π({meridional_term} + "
                       f"{radial_term}) + {phase:.4f})")
            formula_parts.append(term)
            
        return " + ".join(formula_parts)

    def analyze_meridional_symmetry(self):
        """Analyze meridional symmetry in the Fourier components."""
        top_components = self.get_top_components(20)  # Look at more components
        
        # Calculate total power in meridional frequencies
        total_power = np.sum(self.power_spectrum)
        meridional_power = 0
        
        for comp in top_components:
            if comp['meridional_freq'] != 0:
                meridional_power += comp['power']
        
        symmetry_metrics = {
            'meridional_power_ratio': (meridional_power / total_power) * 100,
            'non_symmetric_components': [comp for comp in top_components[:10] 
                                       if comp['meridional_freq'] != 0],
            'is_radially_symmetric': (meridional_power / total_power) < 0.01
        }
        
        return symmetry_metrics

    def reconstruct_surface(self, n_components=5):
        """Reconstruct corneal surface using specified number of components."""
        # Get top components
        components = self.get_top_components(n_components)
        
        # Create reconstruction spectrum
        recon_spectrum = np.zeros_like(self.fft_shifted, dtype=complex)
        
        for comp in components:
            i = comp['meridional_freq'] + self.n_meridians//2
            j = comp['radial_freq'] + self.n_radial//2
            magnitude = comp['magnitude'] 
            phase = comp['phase']
            recon_spectrum[i, j] = magnitude * np.exp(1j * phase)
            
        # Perform inverse FFT and denormalize
        reconstructed = np.real(ifft2(ifftshift(recon_spectrum))) * self.grid_norm
        
        # Calculate error metrics
        error = self.original_grid - reconstructed
        error_metrics = {
            'mse': np.mean(error**2),
            'rmse': np.sqrt(np.mean(error**2)),
            'mae': np.mean(np.abs(error)),
            'max_error': np.max(np.abs(error)),
            'relative_error': np.mean(np.abs(error/self.original_grid)) * 100
        }
        
        return reconstructed, error_metrics

    def get_comprehensive_analysis(self):
        """Get comprehensive analysis of corneal topography."""
        # Basic statistics
        analysis = {
            'grid_dimensions': {
                'meridians': self.n_meridians,
                'radial_points': self.n_radial
            },
            'basic_stats': {
                'dc_component': float(self.dc_component * self.grid_norm),
                'max_magnitude': float(np.max(self.magnitude_spectrum) * self.grid_norm),
                'max_power': float(np.max(self.power_spectrum) * self.grid_norm**2),
                'mean_magnitude': float(np.mean(self.magnitude_spectrum) * self.grid_norm),
                'std_magnitude': float(np.std(self.magnitude_spectrum) * self.grid_norm),
                'total_power': float(np.sum(self.power_spectrum) * self.grid_norm**2)
            }
        }
        
        # Add detailed top components analysis
        analysis['top_components'] = [{
            'rank': comp['rank'],
            'magnitude': comp['magnitude'] * self.grid_norm,
            'meridional_freq': comp['meridional_freq'],
            'radial_freq': comp['radial_freq'],
            'phase': comp['phase'],
            'relative_power': comp['relative_power']
        } for comp in self.get_top_components(5)]
        
        # Add frequency band analysis
        analysis['frequency_bands'] = self.analyze_frequency_bands(5)
        
        # Add symmetry analysis
        analysis['symmetry'] = self.analyze_meridional_symmetry()
        
        # Add reconstruction analysis
        reconstructed, error_metrics = self.reconstruct_surface(5)
        formula = self.get_reconstruction_formula(5)
        
        analysis['reconstruction'] = {
            'formula': formula,
            'error_metrics': error_metrics
        }
        
        return analysis

def main():
    """Example usage"""
    # Create analyzer instance
    analyzer = CornealFourierAnalyzer('/home/aricept094/mydata/testrm_converted.csv')
    
    # Get comprehensive analysis
    analysis = analyzer.get_comprehensive_analysis()
    
    # Print results
    print("\nBasic Statistics:")
    for key, value in analysis['basic_stats'].items():
        print(f"{key}: {value}")
    
    print("\nTop 5 Frequency Components:")
    for comp in analysis['top_components']:
        print(f"\nRank {comp['rank']}:")
        print(f"Magnitude: {comp['magnitude']:.2f}")
        print(f"Meridional Frequency: {comp['meridional_freq']}")
        print(f"Radial Frequency: {comp['radial_freq']}")
        print(f"Phase: {comp['phase']:.4f}")
        print(f"Relative Power: {comp['relative_power']:.2f}%")
    
    print("\nFrequency Band Analysis:")
    for band in analysis['frequency_bands']:
        print(f"\nBand {band['band_index']}:")
        print(f"Frequency Range: {band['frequency_range']}")
        print(f"Relative Power: {band['relative_power']:.2f}%")
        print(f"Decay Rate: {band['decay_rate']:.4f}")
    
    print("\nMeridional Symmetry Analysis:")
    print(f"Power in meridional variations: {analysis['symmetry']['meridional_power_ratio']:.4f}%")
    print(f"Radially symmetric? {analysis['symmetry']['is_radially_symmetric']}")
    
    if analysis['symmetry']['non_symmetric_components']:
        print("\nNon-symmetric components found:")
        for comp in analysis['symmetry']['non_symmetric_components']:
            print(f"Meridional freq: {comp['meridional_freq']}, "
                  f"Relative power: {comp['relative_power']:.4f}%")
    else:
        print("\nNo significant non-symmetric components found in top 10")
    
    print("\nReconstruction Formula:")
    print(analysis['reconstruction']['formula'])
    
    print("\nError Metrics:")
    for metric, value in analysis['reconstruction']['error_metrics'].items():
        print(f"{metric}: {value}")

if __name__ == "__main__":
    main()


Basic Statistics:
dc_component: 319155.02148888004
max_magnitude: 319155.02148888004
max_power: 101859927741.56746
mean_magnitude: 69.3032441560329
std_magnitude: 3565.7204553894576
total_power: 104195402183.74847

Top 5 Frequency Components:

Rank 1:
Magnitude: 319155.02
Meridional Frequency: 0
Radial Frequency: 0
Phase: -0.0000
Relative Power: 97.76%

Rank 2:
Magnitude: 27596.22
Meridional Frequency: 0
Radial Frequency: 1
Phase: -1.8869
Relative Power: 0.73%

Rank 3:
Magnitude: 27596.22
Meridional Frequency: 0
Radial Frequency: -1
Phase: 1.8869
Relative Power: 0.73%

Rank 4:
Magnitude: 12219.68
Meridional Frequency: 0
Radial Frequency: 2
Phase: -1.2818
Relative Power: 0.14%

Rank 5:
Magnitude: 12219.68
Meridional Frequency: 0
Radial Frequency: -2
Phase: 1.2818
Relative Power: 0.14%

Frequency Band Analysis:

Band 0:
Frequency Range: (0.0, 3.2)
Relative Power: 99.74%
Decay Rate: 0.0381

Band 1:
Frequency Range: (3.2, 6.4)
Relative Power: 0.14%
Decay Rate: 0.0015

Band 2:
Frequency Ra